 # Przygotowanie tekstów do analizy za pomocą metody topic modeling 

W naszym eksperymencie posłużymy się dostępnymi za darmo korpusami - zbiorami tekstów - ELTeC (European Literary Text Collection), utworzonymi m.in. przez nasz zespół w ramach COST Action Distant Reading for European Literary History (CA16204, https://distant-reading.net).

Tak jak wiele zasobów literackich, korpusy ELTeC są dostępne w formacie XML, który pozwala na zachowanie różnych informacji m.in. o formie tekstu - podziale na akapity, rozdziały itp. Tymczasem większość metod text miningu preferuje korzystanie z możliwie najprostszych formatów tekstu, przede wszystkim tzw. "plain text" (czyli .txt). Metody modelowania pojęciowego (topic modeling) nie są tu wyjątkiem. Co więcej, ponieważ metoda ta była wynaleziona na tworzenia słów kluczy i tagów na podstawie krótkich tekstów, np. abstraktów, wymaga ona także podziału na stosunkowo krótkie fragmenty tekstu - a więc korpus powieści musimy odpowiednio "pokroić". W naszym przypadku, pokroimy go na akapity.

In [23]:
# Wczytujemy potrzebne nam biblioteki
import lxml
import os
from bs4 import BeautifulSoup
from simplemma import text_lemmatizer

In [67]:
czech_path = os.listdir("ELTeC_originals/Cze/level1")
polish_path = os.listdir("ELTeC_originals/Pol/level1")
cze_output_path = os.listdir("plain_texts/Cze/")
pol_output_path = os.listdir("plain_texts/Pol/")

Możemy sprawdzić, że w zmiennej faktycznie znajduje się lista naszych plików wyświetlając pięć pierwszych elementów w ten sposób:

Mając listę plików przechodzimy do sedna - chcemy usunąć niepotrzebne nam znaczniki XML. Zrobimy to używając funkcji etree z biblioteki lxml. Co istotne, ponieważ oba języki w naszym studium mają znaki diakrytyczne, chcemy się upewnić, że przy ich przetwarzaniu będziemy trzymać się kodowania UTF-8, które poprawnie je zapisuje.

In [87]:
for path, subdirs, files in os.walk("ELTeC_originals/Cze/level1"):
    for name in files:
        file_path = os.path.join(path,name)
# Wczytujemy plik i od razu "wrzucamy go do zupy" zastrzegając, że to xml
        with open(file_path) as file:
            soup = BeautifulSoup(file, 'lxml')
# Znajdujemy tekst oznaczony paragrafami (<p>) w części zawierającej tekst (text)            
        tags = soup.find('text').find_all('p')
# Dodajemy znak nowej linii (\n) pomiędzy wykryte paragrafy
        stripped = ("\n").join([t.text for t in tags])
# Przygotowujemy nową nazwę pliku  
            new_name = os.path.splitext(name)[0]
            new_path = os.path.join("plain_texts/Cze/",new_name+".txt")
# I zapisujemy
            with open(new_path, 'w+') as output:
                output.write(stripped)       


In [89]:
for path, subdirs, files in os.walk("ELTeC_originals/Pol/level1"):
    for name in files:
        file_path = os.path.join(path,name)
# Wczytujemy plik i od razu "wrzucamy go do zupy" zastrzegając, że to xml
        with open(file_path) as file:
            soup = BeautifulSoup(file, 'lxml')
# Znajdujemy tekst oznaczony paragrafami (<p>) w części zawierającej tekst (text)
        tags = soup.find('text').find_all('p')
# Dodajemy znak nowej linii (\n) pomiędzy wykryte paragrafy
        stripped = ("\n").join([t.text for t in tags])
# Przygotowujemy nową nazwę pliku  
        new_name = os.path.splitext(name)[0]
        new_path = os.path.join("plain_texts/Pol/", new_name+".txt")
# I zapisujemy
        with open(new_path, 'w+') as output:
                output.write(stripped) 

Gdy mamy już pliki z czystym tekstem, chcemy poddać je lemmatyzacji - czyli sprowadzić je do formy podstawowej. 

In [14]:
for path, subdirs, files in os.walk("plain_texts/Cze"):
    for name in files:
        file_path = os.path.join(path,name)
        with open(file_path) as file:
            text = file.read()
# Aby zachować strukturę podziału na paragrafy, lematyzację wykonujemy na paragrafach       
        paragraphs = text.split("\n")
        result = [text_lemmatizer(p, lang='cs') for p in paragraphs]
        
# Zlematyzowane paragrafy chcemy połączyć w jeden tekst. Aby zachować informację o końcu paragrafu, w miejsce znaku końca linii podstawiamy znacznik EOL.
        joined_lines = ["\n".join(x) for x in result]
        joined_text = "\nEOL\n".join(joined_lines) 
# Przygotowujemy nową nazwę pliku            
        new_name = os.path.splitext(name)[0]
        new_path = os.path.join("lemmatized_texts/Cze/", new_name+"_lemmatized.txt")
# I zapisujemy
        with open(new_path, 'w+') as output:
            output.write(joined_text)

In [15]:
for path, subdirs, files in os.walk("plain_texts/Pol"):
    for name in files:
        file_path = os.path.join(path,name)
        
        with open(file_path) as file:
            text = file.read()
            
        paragraphs = text.split("\n")
        result = [text_lemmatizer(p, lang="pl") for p in paragraphs]
        joined_lines = ["\n".join(x) for x in result]
        joined_text = "\nEOL\n".join(joined_lines)
        
        new_name = os.path.splitext(name)[0]
        new_path = os.path.join("lemmatized_texts/Pol/", new_name+"_lemmatized.txt")
        with open(new_path, 'w+') as output:
            output.write(joined_text)

Gotowe zlematyzowane teksty możemy teraz poddać wybranej metodzie analizy - w naszym wypadku topic modelingowi.